To use dash and Plotly, make sure to run these commands in a terminal to install the necessary packages:

pip install dash <br>
pip install Plotly <br>
pip install jupyter-dash <br>

Make sure to also install pandas if you need to with this command:

pip install pandas

In [13]:
import plotly
import dash
import jupyter_dash

# This cell should show you what version you have installed when executed
print("Plotly version:", plotly.__version__)
print("Dash version:", dash.__version__)
print("JupyterDash version:", jupyter_dash.__version__)


Plotly version: 5.24.1
Dash version: 2.18.2
JupyterDash version: 0.4.2


In [17]:
import numpy as np
import pandas as pd

print(np.__version__)
print(pd.__version__)


2.1.3
2.2.3


In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Load your data
df = pd.read_csv('super_covid19_data.csv')  # Place holder assuming super covid data

# Reshape the data: melt to have dates and cases in single columns
date_columns = [col for col in df.columns if '_cases' in col]
df_melted = df.melt(id_vars=['countyFIPS', 'County Name_x', 'State', 'StateFIPS_cases'],
                    value_vars=date_columns,
                    var_name='date',
                    value_name='cases')

# Convert date column to datetime and clean up date format
df_melted['date'] = pd.to_datetime(df_melted['date'].str.replace('_cases', ''))

# Filter to a specific state (e.g., NC) or set up the dashboard to allow state selection
# Initializing the Dash app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("COVID-19 Cases by County"),
    
    # Dropdown for selecting a state
    html.Label("Select State:"),
    dcc.Dropdown(
        id='state-dropdown',
        options=[{'label': state, 'value': state} for state in df_melted['State'].unique()],
        value='NC'  # Default to North Carolina, can be any state in your data
    ),
    
    # Date range picker
    html.Label("Select Date Range:"),
    dcc.DatePickerRange(
        id='date-picker-range',
        min_date_allowed=df_melted['date'].min(),
        max_date_allowed=df_melted['date'].max(),
        start_date=df_melted['date'].min(),
        end_date=df_melted['date'].max()
    ),
    
    # Radio items for scale type
    html.Label("Scale:"),
    dcc.RadioItems(
        id='scale-radio',
        options=[{'label': 'Linear', 'value': 'linear'},
                 {'label': 'Log', 'value': 'log'}],
        value='linear',
        labelStyle={'display': 'inline-block'}
    ),
    
    # Main graph
    dcc.Graph(id='covid-cases-graph')
])

# Callback to update the graph based on inputs
@app.callback(
    Output('covid-cases-graph', 'figure'),
    [Input('state-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('scale-radio', 'value')]
)
def update_graph(selected_state, start_date, end_date, scale_type):
    # Filter data based on state and date range
    filtered_df = df_melted[(df_melted['State'] == selected_state) &
                            (df_melted['date'] >= start_date) &
                            (df_melted['date'] <= end_date)]
    
    # Aggregate cases by date
    daily_cases = filtered_df.groupby('date')['cases'].sum().reset_index()

    # Create the figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=daily_cases['date'], y=daily_cases['cases'],
        mode='lines', name='Daily Cases'
    ))
    
    # Apply scale (linear or log)
    fig.update_layout(
        yaxis_type=scale_type,
        title=f"COVID-19 Cases in {selected_state}",
        xaxis_title="Date",
        yaxis_title="Number of Cases"
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
